# Elastic Constants Calculations Based on Strain Fluctuations Method
# 1. Introduction
- In this example, we calculate the elastic constants of diamond Si at 300 K and zero pressure. Based on the strain fluctuations method [[Parrinello 1982]](https://doi.org/10.1063/1.443248), one can directly measure the elastic constants from the fluctuations of unit cell vectors in the constant-stress (N, $\sigma$, T) molecular dynamics simulations:
$$
C_{ijkl}^{-1} = \frac{V}{k_{B}T}\left\langle\epsilon_{ij}\epsilon_{kl}\right\rangle-\left\langle\epsilon_{ij}\right\rangle\left\langle\epsilon_{kl}\right\rangle,
$$
where $\epsilon$ is the unit cell strain, $V$ is the unit cell volume, $T$ is the temperature and $k_{B}$ is the Boltzmann constant. $\left\langle\right\rangle$ represents the ensemble average and calculated as the the average over a period of time in molecular dynamics simulations. 


- In this tutorial, we use the stochastic cell rescaling (scr) ensemble [[Bernetti 2020]](https://doi.org/10.1063/5.0020514), which is a first-order barostat that samples the correct volume fluctuations by including a suitable noise term. The scr ensemble has been implemented in GPUMD recently. However, we stress that the strain fluctuation methods **cannot** be performed using **the Berendsen method**, due to the fact that the Berendsen method cannot give the correct volume fluctuations. 

# 2. Preparing the Inputs
- We use a diamond Si consisting of 8000 atoms. 


- The atomic interactions are described by the minimal Tersoff potential [[Fan 2020]](https://doi.org/10.1088/1361-648X/ab5c5f).

## Generate the  [xyz.in](https://gpumd.zheyongfan.org/index.php/The_xyz.in_input_file) file:

- The xyz.in file for diamond silicon is contructed by the matlab code create_xyz.m as provided in the same path, which is replicated from the [cohesive_energy tutorial](https://github.com/brucefan1983/GPUMD/tree/master/examples/empirical_potentials/cohesive_energy). Because we need to obtain the fluctuation of lattice angle, we use the triclinic box  instead of orthogonal box here.   

## The <code>run.in</code> file:
The <code>run.in</code> input file is given below:<br>
```
potential   potentials/tersoff/Si_Fan_2019.txt 0
velocity    300

ensemble    npt_scr 300 300 100 0 0 0 0 0 0 2000
time_step   1
dump_thermo  100         
run       1100000
```
 - The first line uses the [potential](https://gpumd.zheyongfan.org/index.php/The_potential_keyword) keyword to define the potential to be used, which is specified in the file [Si_Fan_2019.txt](https://github.com/brucefan1983/GPUMD/blob/master/potentials/tersoff/Si_Fan_2019.txt).


 - The second line uses the [velocity](https://gpumd.zheyongfan.org/index.php/The_velocity_keyword) keyword and sets the velocities to be initialized with a temperature of 300 K. 


- For the [run](https://gpumd.zheyongfan.org/index.php/The_run_keyword), the constant-stress (N, $\sigma$, T), specifically, NPT [ensemble](https://gpumd.zheyongfan.org/index.php/The_ensemble_keyword) (the scr method) is used. The temperature is 300 K and the pressures are zero in all the directions. Currently, all the box components will be controlled independently according to the 6 target pressure components. Therefore, we can obtain the fluctuation of all lattice constants including three lattice length and three lattice angles. 


- The coupling constants are 100 (dimensionless) and 2000 (in the natural unit system adopted by GPUMD) for the thermostat and the barostat, respectively. The [time_step](https://gpumd.zheyongfan.org/index.php/The_time_step_keyword) for integration is 1 fs. There are $1.1\times 10^6$ steps (1100 ps) for this [run](https://gpumd.zheyongfan.org/index.php/The_run_keyword) and the thermodynamic quantities will be output every 100 steps. The first 100 ps serves as the equilibration stage, and the last 1000 ps serves as the production stage.


# 3. Results and Discussion
Based on the [thermo.out](https://gpumd.zheyongfan.org/index.php/The_thermo.out_output_file) output files, according to the strain fluctuation method defined above, one can obtain the four-order elastic tensor based on fluctuation of corresponding strain compoments, and then obtain the two-order elastic constants under Voigt notation. Specifically, dimaond Si considered here has a cubic symmestry, there are only three elastic constants, i.e., C11, C12, C44. Based on this simple example, one can be easily extended to the structures with other symmetry.  

In [1]:
from thermo.gpumd.data import load_thermo
from pylab import *
import math

def calc_angle(v1, v2):
    alpha = math.acos(np.dot(v1, v2)/(np.linalg.norm(v1)*np.linalg.norm(v2)))
    return alpha

def calc_elstic(strain_ij, strain_kl):
    s_ijkl = average(strain_ij*strain_kl)-average(strain_ij)*average(strain_kl)
    return s_ijkl

def output_elastic_gpumd(path, slice_num):
    #Obtain the strain tensor     
    thermo = load_thermo(directory=path)
    for keys in thermo:
        thermo[keys]=thermo[keys][1000:] #Discard the first 100 ps data
    Cij = np.zeros((slice_num,3))
    for i in range(slice_num): #Split to 10 slices
        # print(i)
        thermo_slice = dict()
        for keys in thermo:
            thermo_slice[keys]=thermo[keys][1000*i:1000*(i+1)]
        alpha = np.zeros(len(thermo_slice["ax"]))
        beta  = np.zeros_like(alpha)
        gamma = np.zeros_like(alpha)
        for j in range(len(thermo_slice["ax"])):
            va = [thermo_slice["ax"][j], thermo_slice["ay"][j], thermo_slice["az"][j]]
            vb = [thermo_slice["bx"][j], thermo_slice["by"][j], thermo_slice["bz"][j]]
            vc = [thermo_slice["cx"][j], thermo_slice["cy"][j], thermo_slice["cz"][j]]
            alpha[j] = calc_angle(vb, vc)
            beta[j]  = calc_angle(va, vc)
            gamma[j] = calc_angle(va, vb)
           
        strain = dict()
        strain["11"] = thermo_slice["ax"]/average(thermo_slice["ax"])-1 #xx 
        strain["22"] = thermo_slice["by"]/average(thermo_slice["by"])-1 #yy
        strain["33"] = thermo_slice["cz"]/average(thermo_slice["cz"])-1 #zz
        strain["23"] = (alpha-pi/2)/2 #yz
        strain["13"] = (beta-pi/2)/2 #xz
        strain["12"] = (gamma-pi/2)/2 #xy
        
        #Calculate the cubic compliance tensor
        #S11=S1111=S2222=S3333,S12=S1122=S1133=S2233; S44=S2323=S1313=S1212
        V = average(thermo["ax"]*thermo["by"]*thermo["cz"]) #volue, unit in angstrom^{3}
        T = 300 #temperature, unit in K
        kB = 1.38064852 #unit in e-23 J/K
        scale = 100/(T*kB)*V #unit in GPa^{-1}, scale=V/(KB*T)
        S1111 = scale*calc_elstic(strain["11"], strain["11"])
        S2222 = scale*calc_elstic(strain["22"], strain["22"])
        S3333 = scale*calc_elstic(strain["33"], strain["33"])
        S1122 = scale*calc_elstic(strain["11"], strain["22"])
        S1133 = scale*calc_elstic(strain["11"], strain["33"])
        S2233 = scale*calc_elstic(strain["22"], strain["33"])
        S2323 = scale*calc_elstic(strain["23"], strain["23"])
        S1313 = scale*calc_elstic(strain["13"], strain["13"])
        S1212  = scale*calc_elstic(strain["12"], strain["12"])
        
        # All the below value should be very close to zero
        S1123 = scale*calc_elstic(strain["11"], strain["23"])
        S1113 = scale*calc_elstic(strain["11"], strain["13"])
        S1112 = scale*calc_elstic(strain["11"], strain["12"])
        
        # Convert Sijkl to  Spq 
        # NOTED that the Spq = Sijkl when p q = 1, 2, 3 BUT Spq = 4*Sijkl when p q = 4, 5, 6.
        S11 = (S1111+S2222+S3333)/3
        S12 = (S1122+S1133+S2233)/3
        S44 = 4*(S2323+S1313+S1212)/3
        Spq = np.array([[S11, S12, S12,   0,   0,   0], 
                        [S12, S11, S12,   0,   0,   0],
                        [S12, S12, S11,   0,   0,   0],
                        [0,     0,   0, S44,   0,   0],
                        [0,     0,   0,   0, S44,   0],
                        [0,     0,   0,   0,   0, S44]])
        
        # Convert Spq to Cpq and 
        Cpq = np.linalg.inv(Spq)
        C11 = Cpq[1,1]
        C12 = Cpq[1,2]
        C44 = Cpq[4,4]
        # print(np.array([C11, C12, C44]))
        Cij[i] = np.array([C11, C12, C44])
    return Cij

def calc_ste(array): # calculate the standard error
    ste = np.zeros(3)
    for i in range(array.shape[1]):
        ste[i] = sqrt(mean(abs(array[:,i] - array[:,i].mean())**2))/len(array[:,i])
    return ste
        

Cij_Si_gpumd_1ns = output_elastic_gpumd("./", 10)
print("[C11, C12, C44] (unit in GPa) of Si estimated by gpumd with npt_scr for 1ns:")
print(np.average(Cij_Si_gpumd_1ns, axis=0))
print("With standard error (unit in GPa) of: ")
print(calc_ste(Cij_Si_gpumd_1ns))

[C11, C12, C44] (unit in GPa) of Si estimated by gpumd with npt_scr for 1ns:
[153.99128892  65.06584192  77.93512353]
With standard error (unit in GPa) of: 
[1.26584141 1.71782377 0.72885593]


- Noted that we evenly divide the 1 ns simulation to ten independent simulations of 100 ps. Based on each independent simulation, we can obtain three elastic constants, and the final elastic constants is obtained averaging on ten simulations. We also compute the corresponding standard deviation.


- According to the [cohesive_energy tutorial](https://github.com/brucefan1983/GPUMD/tree/master/examples/empirical_potentials/cohesive_energy), one can also straightly calculate the elastic constants at 0 K based on stress-strain relationship by deforming the box with small strain. The elastic constants at 0 K are calculated as C11 = 148.686 Gpa, C12 = 65.5985 GPa ,C44 = 74.9417 GPa, which is in good agreement with our result obtained from strain fluctuation method at 300 K. 


- Rather than the case at finite temperature, the strain fluctuation method can also been extended to the case under pressure loading. Combined with the Born stability criteria [[Born 1954]](https://en.wikipedia.org/wiki/Dynamical_Theory_of_Crystal_Lattices), strain fluctuation method is a powerful method to investigate the mechanical stability under pressure loading. 


# 4. References
- [Parrinello 1982] M. Parrinello and A. Rahman. [Strain fluctuations and elastic constants](https://doi.org/10.1063/1.443248), J. Chem. Phys. **76**, 2662 (1982).
- [Bernetti 2020] Mattia Bernetti and  Giovanni Bussia, [Pressure control using stochastic cell rescaling](https://doi.org/10.1063/5.0020514), J. Chem. Phys. **153**, 114107 (2020).
- [Fan 2020] Zheyong Fan, Yanzhou Wang, Xiaokun Gu, Ping Qian, Yanjing Su, and Tapio Ala-Nissila, [A minimal Tersoff potential for diamond silicon with improved descriptions of elastic and phonon transport properties](https://doi.org/10.1088/1361-648X/ab5c5f), J. Phys.: Condens. Matter 32 135901 (2020).
- [Born 1954] M. Born, K. Huang, Dynamical Theory of Crystal Lattices, Clarendon Press, 1954.

